In [1]:
import os
import requests
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from functools import partial

# We use the alpha-vantage package to pull historical CAD data
# Install package before importing: pip install alpha-vantage
# https://pypi.org/project/alpha-vantage/
from alpha_vantage.foreignexchange import ForeignExchange



In [2]:
# #########################################################
#  Pull Historical CAD data from alpha vantage API
# #########################################################

# Load .env enviroment variables
load_dotenv()
alpha_vantage_key = os.getenv("ALPHA_VANTAGE")

#Pull CAD Data
fx = ForeignExchange(key=alpha_vantage_key,output_format='pandas')
cad_data, meta_data = fx.get_currency_exchange_daily(from_symbol='CAD', to_symbol='USD', outputsize='full')

# Save to CSV file
cad_data.to_csv('../../02-Data/Dirty Data/Market Data/CAD_5yr_drt.csv')

# *** Format and clean CAD data ***

# Keep only Date and Close columns
cad_data.reset_index(inplace=True)
cad_data.drop(['1. open', '2. high', '3. low'], axis=1, inplace=True)

# Rename columns
cad_data.columns = ['Date', 'Close']

# Select only from 2015-10-01 to 2020-11-01
start_date = pd.Timestamp("10/31/2015")
end_date = pd.Timestamp("11/01/2020")

# Set Date column to 'Timestamp' format
cad_data['Date'] = pd.to_datetime(cad_data['Date'])

# Slice Dataframe
cad_data = cad_data[(cad_data['Date'] > (start_date)) & (cad_data['Date'] < (end_date))]

# Set index back
cad_data.set_index("Date", inplace = True)

# Sort Index
cad_data = cad_data.sort_index(ascending=True)

# Save to file
cad_data.to_csv('../../02-Data/Clean Data/Market Data/CAD_5yr_cln.csv')

cad_data.head()

,Close
Date,
2015-11-02,0.7632
2015-11-03,0.7652
2015-11-04,0.7601
2015-11-05,0.7591
2015-11-06,0.7515


In [3]:
# ########################################################################
#  Pull Historical SP-TSX60 data from CSV file pulled from investing.com
# ########################################################################

# Clean TSX60 Dataframe
# TSX from 2015-10-01 to 2020-11-01
SP_TSX500_file = Path("../../02-Data/Dirty Data/Market Data/SPTSX60 Historical.csv")
SP_TSX500_data = pd.read_csv(SP_TSX500_file, index_col="Date")

# Remove useles column
SP_TSX500_data.reset_index(inplace=True)

# Convert Date column to datetime type
SP_TSX500_data['Date'] = pd.to_datetime(SP_TSX500_data['Date'])

# Change column order and keep only relevant ones
SP_TSX500_data_sliced = SP_TSX500_data[['Date', 'Price']].set_index('Date')

# Save to CSV file
SP_TSX500_data_sliced.to_csv('../../02-Data/Clean Data/Market Data/SPTSX60_5yr_cln.csv')

SP_TSX500_data_sliced.head()

,Price
Date,
2020-10-30,928.90
2020-10-29,934.83
2020-10-28,930.90
2020-10-27,954.81
2020-10-26,959.91
